In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pylab as plt
import plotly.graph_objects as go
from typing import Tuple
import vectorbt as vbt

plt.rc('text', usetex=True)
plt.rc('font',**{'family':'sans-serif','serif':['Palatino']})
figSize  = (12, 8)
fontSize = 20

## Acknowledgement 

Credit to [kridtapon](https://medium.com/@kridtapon) and learning from his posts on Medium.


## Data 

In [2]:
%%capture
ticker = 'HSBA.L'
data = yf.download(ticker, start='2010-01-01', end='2025-01-14')

In [3]:
data_clean = data.copy()
data_clean = data_clean.ffill()
data_clean.columns = [data.columns.values[i][0] for i in range(len(data.columns.values))]
df = data_clean.copy()

In [4]:
df.head()

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2010-01-04,333.607147,726.500000,726.500000,713.000000,713.299988,15139314
2010-01-05,338.428558,737.000000,747.200012,723.000000,723.000000,27480532
2010-01-06,339.989868,740.400024,744.000000,734.200012,737.099976,17203196
2010-01-07,338.199097,736.500000,742.400024,730.099976,735.099976,26192632
2010-01-08,339.806335,740.000000,742.500000,729.000000,740.200012,20725091


In [5]:
df.tail()

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2025-01-07,776.000000,776.000000,776.909973,767.000000,774.500000,19327480
2025-01-08,791.200012,791.200012,792.427002,781.400024,782.700012,23420002
2025-01-09,802.500000,802.500000,802.500000,790.099976,794.099976,61105458
2025-01-10,799.099976,799.099976,802.900024,790.299988,800.599976,18662771
2025-01-13,798.299988,798.299988,799.500000,788.400024,794.599976,36198140


In [6]:
nrecent = 100
fig = go.Figure(data=[go.Candlestick(
    x=df.index[-nrecent:],
    open=df['Open'][-nrecent:],
    high=df['High'][-nrecent:],
    low=df['Low'][-nrecent:],
    close=df['Close'][-nrecent:]
)])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_layout(
    title=f'{ticker} Stock Price',
    title_x=0.5,  # Center the title
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly_dark'
)
fig.show()

### 1. Chande Momentum Oscillator (CMO)

The Chande Momentum Oscillator (CMO) is a technical indicator used in trading to measure momentum by comparing the sum of gains and losses over a specified period. It oscillates between -100 and +100, where positive values indicate bullish momentum and negative values indicate bearish momentum. The formula is:

$$
\textrm{CMO} = 100\times \dfrac{S_{\textrm{up}} - S_{\textrm{down}}}{S_{\textrm{up}} + S_{\textrm{down}}}
$$

where 

- $S_{\textrm{up}}$ is the sum of the upward price changes over the period and 
- $S_{\textrm{down}}$ is the sum of downward price changes over the period.

In [7]:
def chande_momentum_oscillator(df: pd.DataFrame, period: int=14) -> pd.Series:
    """
    Calculate the Chande Momentum Oscillator (CMO) for a given DataFrame.

    The CMO measures momentum by comparing the sum of gains and losses over a specified period.
    It oscillates between -100 and +100, where positive values indicate bullish momentum and
    negative values indicate bearish momentum.

    Args:
        df (pd.DataFrame): A DataFrame containing a 'Close' column with closing prices.
        period (int, optional): The number of periods to calculate the CMO. Default is 14.

    Returns:
        pd.Series: A Series containing the CMO values for each row in the DataFrame.

    Example:
        >>> import pandas as pd
        >>> data = {'Close': [10, 12, 11, 15, 14]}
        >>> df = pd.DataFrame(data)
        >>> cmo = chande_momentum_oscillator(df, period=3)
        >>> print(cmo)
    """
    # Calculate upward and downward movements
    df['Up'] = np.where(df['Close'] > df['Close'].shift(1), df['Close'] - df['Close'].shift(1), 0)
    df['Down'] = np.where(df['Close'] < df['Close'].shift(1), df['Close'].shift(1) - df['Close'], 0)

    # Sum of upward and downward movements over the period
    df['SumUp'] = df['Up'].rolling(window=period).sum()
    df['SumDown'] = df['Down'].rolling(window=period).sum()

    # Calculate CMO using the formula
    df['CMO'] = 100 * (df['SumUp'] - df['SumDown']) / (df['SumUp'] + df['SumDown'])

    # Return the CMO values
    return df['CMO']


In [8]:
df['CMO'] = chande_momentum_oscillator(df)
df.dropna(inplace=True)

In [9]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['CMO'], mode='lines', name='CMO'))
fig.update_layout(title='Chande Momentum Oscillator (CMO)',
                  xaxis_title='Time',
                  yaxis_title='Value',
                  template='plotly_dark')
fig.show()

In [10]:
def cmo_trading_signals(cmo: pd.Series, overbought: float = 50, oversold: float = -50) -> pd.Series:
    """
    Generate trading signals (Buy, Hold, or Sell) based on the Chande Momentum Oscillator (CMO).

    The function analyzes the given CMO values to determine whether the market is in an overbought
    or oversold condition and generates trading signals accordingly:

    - **Buy Signal (1):** When the CMO crosses above the oversold threshold (e.g., -50).
    - **Sell Signal (-1):** When the CMO crosses below the overbought threshold (e.g., +50).
    - **Hold Signal (0):** When the CMO does not cross either threshold.

    Args:
        cmo (pd.Series): A Pandas Series representing the CMO values for a time series.
                         Typically calculated using a financial time series with closing prices.
        overbought (float, optional): The threshold indicating overbought conditions.
                                      Default is 50.
        oversold (float, optional): The threshold indicating oversold conditions.
                                    Default is -50.

    Returns:
        pd.Series: A Pandas Series of trading signals corresponding to the input CMO Series:
                   - 1: Buy Signal
                   - -1: Sell Signal
                   - 0: Hold Signal

    Example:
        ```python
        import pandas as pd

        # Example DataFrame with CMO values
        data = {'CMO': [-60, -55, -45, 5, 55, 45, -10, -60]}
        df = pd.DataFrame(data)

        # Generate trading signals
        signals = cmo_trading_signals(df['CMO'])

        print(signals)
        ```
        Output:
            0    0
            1    1
            2    0
            3    0
            4   -1
            5    0
            6    0
            7    0
            dtype: int64

    Notes:
        - The thresholds for overbought and oversold conditions can be customized based on the
          volatility of the market or the asset being traded.
        - A crossover-based approach is used to avoid false signals from static CMO values
          within the overbought or oversold ranges.
        - Ensure the input CMO Series is properly pre-computed before passing to this function.
    """
    signals = pd.Series(0, index=cmo.index)  # Default to Hold (0)

    # Buy Signal: CMO crosses above the oversold threshold
    buy_signal = (cmo > oversold) & (cmo.shift(1) <= oversold)

    # Sell Signal: CMO crosses below the overbought threshold
    sell_signal = (cmo < overbought) & (cmo.shift(1) >= overbought)

    # Assign signals
    signals[buy_signal] = 1  # Buy
    signals[sell_signal] = -1  # Sell

    return signals

## 2. Klinger Volume Oscillator (KVO)

The Klinger Volume Oscillator (KVO) is a momentum indicator that uses volume and price movements to identify trends and reversals. It calculates the difference between short-term and long-term moving sums of the Volume Force, which is weighted by True Range. A signal line is derived by smoothing the KVO.

- True Range (TR): Captures the range of price movements.

$$
\textrm{TR} = \textrm{max}(\textrm{High} - \textrm{Low},|\textrm{High} - \textrm{Close}_{\textrm{prev}}|,|\textrm{Low}-\textrm{Close}_{\textrm{prev}}|)
$$

- Volume Force (VF): Measures the impact of volume on price changes.

$$
\textrm{VF}=\begin{cases}
\begin{array}{c}
\textrm{Volume}\\
-\textrm{Volume}
\end{array} & \begin{array}{c}
\textrm{if Close}>\textrm{Close}_{\textrm{prev}}\\
\textrm{if Close}\leq\textrm{Close}_{\textrm{prev}}
\end{array}\end{cases}
$$

- KVO: Differentiates short-term and long-term momentum in volume trends.

$$
\textrm{KVO=\textrm{RollingSum}}_{\textrm{short}}(\textrm{VF})-\textrm{RollingSum}_{\textrm{long}}(\textrm{VF})
$$

- Signal Line: Smooths the KVO for better interpretation of trends.

$$
\textrm{KVO}_{\textrm{Signal}}=\textrm{RollingMean}_{\textrm{signal}}(\textrm{KVO})
$$

In [11]:

def klinger_volume_oscillator(
    df: pd.DataFrame,
    short_period: int = 34,
    long_period: int = 55,
    signal_period: int = 13
) -> Tuple[pd.Series, pd.Series]:
    """
    Calculate the Klinger Volume Oscillator (KVO) and its Signal Line.

    The KVO is a momentum indicator that uses volume and price movements to identify
    trends and reversals. It computes the difference between short-term and long-term
    moving sums of the Volume Force (VF), which is weighted by True Range (TR).

    Args:
        df (pd.DataFrame): A DataFrame containing columns 'High', 'Low', 'Close', and 'Volume'.
        short_period (int, optional): The short-term period for calculating the KVO. Default is 34.
        long_period (int, optional): The long-term period for calculating the KVO. Default is 55.
        signal_period (int, optional): The period for smoothing the signal line. Default is 13.

    Returns:
        Tuple[pd.Series, pd.Series]: A tuple containing:
            - The KVO values as a Pandas Series.
            - The Signal Line values as a Pandas Series.

    Example:
        >>> data = {'High': [12, 14, 13], 'Low': [10, 11, 12],
        >>>         'Close': [11, 13, 12], 'Volume': [1000, 1100, 900]}
        >>> df = pd.DataFrame(data)
        >>> kvo, signal = klinger_volume_oscillator(df)
        >>> print(kvo, signal)
    """
    # Calculate True Range (TR)
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = np.abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = np.abs(df['Low'] - df['Close'].shift(1))
    df['TrueRange'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)

    # Calculate Volume Force (VF)
    df['VolumeForce'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], -df['Volume'])

    # Calculate KVO
    df['KVO'] = (
        df['VolumeForce'].rolling(window=short_period).sum() -
        df['VolumeForce'].rolling(window=long_period).sum()
    )

    # Calculate Signal Line
    df['KVO_Signal'] = df['KVO'].rolling(window=signal_period).mean()

    return df['KVO'], df['KVO_Signal']


In [12]:
df['KVO'], df['KVO_Signal'] = klinger_volume_oscillator(df)
df.dropna(inplace=True)

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['KVO'], mode='lines', name='KVO'))
fig.add_trace(go.Scatter(x=df.index, y=df['KVO_Signal'], mode='lines', name='KVO Signal'))
fig.update_layout(title='Klinger Volume Oscillator (KVO)',
                  xaxis_title='Time',
                  yaxis_title='Value',
                  template='plotly_dark')
fig.show()

In [14]:
def kvo_trading_signals(kvo: pd.Series, signal: pd.Series) -> pd.Series:
    """
    Generate buy/hold/sell trading signals based on Klinger Volume Oscillator (KVO).

    Args:
        kvo (pd.Series): The KVO values.
        signal (pd.Series): The signal line values for the KVO.

    Returns:
        pd.Series: Trading signals (1 = Buy, -1 = Sell, 0 = Hold).
    """
    signals = pd.Series(0, index=kvo.index)  # Default to Hold (0)

    # Buy Signal: KVO crosses above the Signal Line
    buy_signal = (kvo > signal) & (kvo.shift(1) <= signal.shift(1))

    # Sell Signal: KVO crosses below the Signal Line
    sell_signal = (kvo < signal) & (kvo.shift(1) >= signal.shift(1))

    # Assign signals
    signals[buy_signal] = 1  # Buy
    signals[sell_signal] = -1  # Sell

    return signals

## 3. Elder’s Force Index (EFI)

The Elder Force Index (EFI) measures the strength of price movements using price change and volume. It indicates the power behind market moves and helps identify trend strength and potential reversals. The smoothed EFI is obtained by applying a moving average over a chosen period.

$$
\textrm{EFI} = (\textrm{Close}_{t}-\textrm{Close}_{t-1})\cdot \textrm{Volume}_{t}
$$

$$
\textrm{EFI}_{\textrm{smoothed}} = \textrm{MA}_{\textrm{period}}(\textrm{EFI})
$$

In [15]:
def elder_force_index(df: pd.DataFrame, period: int = 2) -> pd.Series:
    """
    Calculate the Elder Force Index (EFI) and its smoothed version.

    The Elder Force Index (EFI) measures the strength of price movements
    by considering price changes and volume. A moving average is applied
    to smooth the EFI values over the specified period.

    Args:
        df (pd.DataFrame): DataFrame containing at least 'Close' and 'Volume' columns.
        period (int, optional): The smoothing period for the EFI. Default is 2.

    Returns:
        pd.Series: A Pandas Series of smoothed EFI values.

    Example:
        ```python
        import pandas as pd

        # Example DataFrame with price and volume data
        data = {'Close': [100, 102, 101, 104], 'Volume': [1000, 1500, 1200, 1300]}
        df = pd.DataFrame(data)

        # Calculate smoothed EFI
        efi_smoothed = elder_force_index(df, period=2)
        print(efi_smoothed)
        ```
    """
    df['EFI'] = (df['Close'] - df['Close'].shift(1)) * df['Volume']
    df['EFI_Smoothed'] = df['EFI'].rolling(window=period).mean()
    return df['EFI_Smoothed']

In [16]:
df['EFI'] = elder_force_index(df)
df.dropna(inplace=True)

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['EFI'], mode='lines', name='EFI'))
fig.update_layout(title="Elder's Force Index (EFI)",
                  xaxis_title='Time',
                  yaxis_title='Value',
                  template='plotly_dark')
fig.show()

In [18]:
def efi_trading_signals(efi: pd.Series, threshold: float = 0) -> pd.Series:
    """
    Generate buy/hold/sell trading signals based on the Elder Force Index (EFI).

    Signals are generated as follows:
    - Buy (1): EFI crosses above the threshold from below.
    - Sell (-1): EFI crosses below the threshold from above.
    - Hold (0): No crossover.

    Args:
        efi (pd.Series): A Pandas Series representing the EFI values.
        threshold (float, optional): Threshold for generating signals. Default is 0.

    Returns:
        pd.Series: A Pandas Series of trading signals:
                   - 1: Buy Signal
                   - -1: Sell Signal
                   - 0: Hold Signal

    Example:
        ```python
        # Generate trading signals using EFI
        signals = efi_trading_signals(efi_smoothed, threshold=0)
        print(signals)
        ```
    """
    signals = pd.Series(0, index=efi.index)  # Default to Hold (0)

    # Buy Signal: EFI crosses above the threshold
    buy_signal = (efi > threshold) & (efi.shift(1) <= threshold)

    # Sell Signal: EFI crosses below the threshold
    sell_signal = (efi < threshold) & (efi.shift(1) >= threshold)

    # Assign signals
    signals[buy_signal] = 1  # Buy
    signals[sell_signal] = -1  # Sell

    return signals

## 4. True Strength Index (TSI)

The True Strength Index (TSI) is a momentum oscillator that evaluates the strength of price trends by smoothing price changes and their absolute values using exponential moving averages (EMAs). It oscillates between positive and negative values, identifying overbought/oversold conditions and momentum shifts.

- **Change in Price**

$$
\textrm{Change}_{t}=\textrm{Close}_{t}-\textrm{Close}_{t-1}
$$

- **Smoothed Price Change**

$$
\textrm{Smoothed}_{1} = \textrm{EMA}_{\textrm{short}}(\textrm{Change})
$$

$$
\textrm{Smoothed}_{2} = \textrm{EMA}_{\textrm{long}}(\textrm{Smoothed}_{1})
$$

- **Smoothed Absolute Price Change**

$$
\textrm{AbsChange}_{t}=|\textrm{Change}_{t}|
$$

$$
\textrm{AbsSmoothed}_{1} = \textrm{EMA}_{\textrm{short}}(\textrm{AbsChange})
$$

$$
\textrm{AbsSmoothed}_{2} = \textrm{EMA}_{\textrm{long}}(\textrm{AbsSmoothed}_{1})
$$

- **True Strength Index**

$$
\textrm{TSI} = \left(\dfrac{\textrm{Smoothed}_{2}}{\textrm{AbsSmoothed}_{2}}\right)\times 100
$$

In [19]:
def true_strength_index(df: pd.DataFrame, short_period: int = 13, long_period: int = 25) -> pd.Series:
    """
    Calculate the True Strength Index (TSI) for a given dataset.

    The True Strength Index is a momentum oscillator that evaluates the strength
    of price trends using smoothed price changes and their absolute values. It
    oscillates between positive and negative values, helping identify trend strength
    and reversals.

    Args:
        df (pd.DataFrame): A DataFrame containing at least a 'Close' column for price data.
        short_period (int, optional): The short EMA smoothing period. Default is 13.
        long_period (int, optional): The long EMA smoothing period. Default is 25.

    Returns:
        pd.Series: A Pandas Series containing the TSI values.

    Example:
        ```python
        import pandas as pd

        data = {'Close': [100, 102, 104, 103, 105]}
        df = pd.DataFrame(data)

        tsi = true_strength_index(df)
        print(tsi)
        ```
    """
    df['Change'] = df['Close'] - df['Close'].shift(1)
    df['Smoothed1'] = df['Change'].ewm(span=short_period, adjust=False).mean()
    df['Smoothed2'] = df['Smoothed1'].ewm(span=long_period, adjust=False).mean()
    df['AbsChange'] = abs(df['Change'])
    df['AbsSmoothed1'] = df['AbsChange'].ewm(span=short_period, adjust=False).mean()
    df['AbsSmoothed2'] = df['AbsSmoothed1'].ewm(span=long_period, adjust=False).mean()
    df['TSI'] = (df['Smoothed2'] / df['AbsSmoothed2']) * 100
    return df['TSI']


In [20]:
df['TSI'] = true_strength_index(df)
df.dropna(inplace=True)

In [21]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['TSI'], mode='lines', name='TSI'))
fig.update_layout(title='True Strength Index (TSI)',
                  xaxis_title='Time',
                  yaxis_title='Value',
                  template='plotly_dark')
fig.show()

In [22]:
def tsi_trading_signals(tsi: pd.Series, overbought: float = 25, oversold: float = -25) -> pd.Series:
    """
    Generate buy/hold/sell trading signals based on the True Strength Index (TSI).

    Signals:
    - Buy (1): TSI crosses above the oversold threshold from below.
    - Sell (-1): TSI crosses below the overbought threshold from above.
    - Hold (0): No crossover.

    Args:
        tsi (pd.Series): A Pandas Series containing the TSI values.
        overbought (float, optional): Overbought threshold. Default is 25.
        oversold (float, optional): Oversold threshold. Default is -25.

    Returns:
        pd.Series: A Pandas Series of trading signals:
                   - 1: Buy Signal
                   - -1: Sell Signal
                   - 0: Hold Signal

    Example:
        ```python
        signals = tsi_trading_signals(tsi, overbought=25, oversold=-25)
        print(signals)
        ```
    """
    signals = pd.Series(0, index=tsi.index)  # Default to Hold (0)

    # Buy Signal: TSI crosses above oversold level
    buy_signal = (tsi > oversold) & (tsi.shift(1) <= oversold)

    # Sell Signal: TSI crosses below overbought level
    sell_signal = (tsi < overbought) & (tsi.shift(1) >= overbought)

    # Assign signals
    signals[buy_signal] = 1  # Buy
    signals[sell_signal] = -1  # Sell

    return signals

## 5. Williams %R

The Williams %R is a momentum oscillator that measures overbought and oversold levels by comparing the current closing price to the high and low prices over a specified period. It oscillates between -100 and 0, with values closer to -100 indicating oversold conditions and values closer to 0 indicating overbought conditions.

- **Highest High Over Period**

$$
\text{High}_{\text{max},t}=\text{max}(\text{High}_{t-1},\text{High}_{t-1},\ldots,\text{High}_{t-\text{period}})
$$

- **Lowest Low Over Period**

$$
\text{Low}_{\text{min},t}=\text{min}(\text{Low}_{t-1},\text{Low}_{t-2},\ldots,\text{Low}_{t-\text{period}})
$$

- **Williams %R**
$$
\text{Williams\;\%R}_t = -100\times \left( \frac{\text{High}_{\text{max}, t} - \text{Close}_t}{\text{High}_{\text{max}, t} - \text{Low}_{\text{min}, t}} \right)
$$

In [23]:
def williams_percent_r(df: pd.DataFrame, period: int = 14) -> pd.Series:
    """
    Calculate the Williams %R for a given dataset.

    Williams %R is a momentum oscillator that measures overbought and oversold
    levels by comparing the current closing price to the high and low prices
    over a specified period.

    Args:
        df (pd.DataFrame): A DataFrame containing at least 'High', 'Low', and 'Close' columns.
        period (int, optional): The lookback period for calculating Williams %R. Default is 14.

    Returns:
        pd.Series: A Pandas Series containing the Williams %R values.

    Example:
        ```python
        import pandas as pd

        data = {'High': [10, 11, 12], 'Low': [8, 9, 10], 'Close': [9, 10, 11]}
        df = pd.DataFrame(data)

        williams_r = williams_percent_r(df)
        print(williams_r)
        ```
    """
    df['High_Max'] = df['High'].rolling(window=period).max()
    df['Low_Min'] = df['Low'].rolling(window=period).min()
    df['Williams_%R'] = ((df['High_Max'] - df['Close']) / (df['High_Max'] - df['Low_Min'])) * -100
    return df['Williams_%R']

In [24]:
df['Williams_%R'] = williams_percent_r(df)
df.dropna(inplace=True)

In [25]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df['Williams_%R'], mode='lines', name='Williams %R'))
fig.update_layout(title='Williams %R',
                  xaxis_title='Time',
                  yaxis_title='Value',
                  template='plotly_dark')
fig.show()

In [26]:
def generate_williams_r_signals(
    williams_r: pd.Series, overbought: float = -20, oversold: float = -80
) -> pd.Series:
    """
    Generate trading signals (buy/hold/sell) based on the Williams %R momentum oscillator.

    Trading Signals:
    - Buy Signal (1): Williams %R crosses above the oversold threshold.
    - Sell Signal (-1): Williams %R crosses below the overbought threshold.
    - Hold Signal (0): No crossover occurs.

    Args:
        williams_r (pd.Series): A Pandas Series containing Williams %R values.
        overbought (float, optional): Overbought threshold level (default: -20).
        oversold (float, optional): Oversold threshold level (default: -80).

    Returns:
        pd.Series: A Pandas Series with trading signals:
                   - 1 for Buy
                   - -1 for Sell
                   - 0 for Hold.

    Example:
        ```python
        # Assuming williams_r is a Pandas Series with Williams %R values
        signals = generate_williams_r_signals(williams_r)
        print(signals)
        ```
    """
    # Initialize signals with default Hold (0)
    signals = pd.Series(0, index=williams_r.index, dtype=int)

    # Generate Buy signals (Williams %R crosses above the oversold threshold)
    buy_signals = (williams_r > oversold) & (williams_r.shift(1) <= oversold)

    # Generate Sell signals (Williams %R crosses below the overbought threshold)
    sell_signals = (williams_r < overbought) & (williams_r.shift(1) >= overbought)

    # Assign signals
    signals[buy_signals] = 1  # Buy
    signals[sell_signals] = -1  # Sell

    return signals

## Entry and Exit Signals

In [27]:
def signal_to_action(signal, columns = ['buy', 'sell']):
    action = pd.DataFrame(np.zeros((signal.shape[0], 2)),
                          index=signal.index,
                          columns=columns,
                          dtype=int)
    action.loc[signal == 1, columns[0]] = 1
    action.loc[signal == -1, columns[1]] = -1
    return action

In [28]:
def enter_exit_signal(dataframe):
    signal_williams_r = generate_williams_r_signals(dataframe['Williams_%R'])
    signal_tsi = tsi_trading_signals(dataframe['TSI'])
    signal_efi = efi_trading_signals(dataframe['EFI'])
    signal_kvo = kvo_trading_signals(dataframe['KVO'], dataframe['KVO_Signal'])
    signal_cmo = cmo_trading_signals(dataframe['CMO'])

    action_williams_r = signal_to_action(signal_williams_r, columns =['Williams_%R_buy', 'Williams_%R_sell'])
    action_tsi = signal_to_action(signal_tsi, columns=['TSI_buy', 'TSI_sell'])
    action_efi = signal_to_action(signal_efi, columns=['EFI_buy', 'EFI_sell'])
    action_kvo = signal_to_action(signal_kvo, columns=['KVO_buy', 'KVO_sell'])
    action_cmo = signal_to_action(signal_cmo, columns=['CMO_buy', 'CMO_sell'])

    actions = pd.concat([action_williams_r, action_tsi, action_efi, action_kvo, action_cmo], axis=1)
    return actions


## Backtesting

Williams %R seems to be the most straightforward indicator to use. 

In [29]:
indicators = ['Williams_%R', 'TSI', 'EFI', 'KVO', 'CMO']
indicator = indicators[0]
actions = enter_exit_signal(df)

In [30]:

date_range =(df.index.year >= 2023) & (df.index.year <= 2025)
df_test = df[date_range]
entries = actions[date_range][f'{indicator}_buy'].to_numpy()
exits = actions[date_range][f'{indicator}_sell'].to_numpy()

In [31]:
# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df_test['Close'],
    entries=entries,
    exits=exits,
    init_cash=1_000_000,
    fees=0.001,
    freq='D',
    # accumulate=True # this makes the backtest accumulate the position size
)

In [32]:
portfolio.positions.records

,id,col,size,entry_idx,entry_price,entry_fees,exit_idx,exit_price,exit_fees,pnl,return,direction,status,parent_id
0,0,0,1731.370882,50,577.000000,999.000999,68,562.400024,973.723026,-27250.696635,-0.027278,0,1,0
1,1,0,1612.373496,103,602.700012,971.777526,107,611.000000,985.160206,11425.742604,0.011758,0,1,1
2,2,0,1627.531690,119,604.099976,983.191854,127,610.400024,993.445383,8276.891878,0.008418,0,1,2
3,3,0,1610.036533,131,615.799988,991.460477,146,636.900024,1025.432307,31954.937023,0.032230,0,1,3
4,4,0,1717.951100,164,595.700012,1023.383491,188,639.799988,1099.145093,73639.072994,0.071956,0,1,4
5,5,0,1800.934235,206,609.099976,1096.948999,221,608.799988,1096.408741,-2733.616026,-0.002492,0,1,5
6,6,0,1817.336253,231,602.099976,1094.218114,236,612.400024,1112.936766,16511.497266,0.015090,0,1,6
7,7,0,1859.867872,265,597.200012,1110.713116,273,611.200012,1136.751266,23790.685829,0.021419,0,1,7
8,8,0,1899.665123,289,597.200012,1134.480035,312,615.000000,1168.294051,31511.241922,0.027776,0,1,8
9,9,0,1681.511041,351,693.400024,1165.959797,370,683.700012,1149.649119,-18626.286543,-0.015975,0,1,9


In [33]:
pf_stats = portfolio.stats()
# pf_stats.to_dict()

In [34]:
pf_stats

Start                         2023-01-03 00:00:00
End                           2025-01-13 00:00:00
Period                          513 days 00:00:00
Start Value                             1000000.0
End Value                          1223998.772439
Total Return [%]                        22.399877
Benchmark Return [%]                    50.651057
Max Gross Exposure [%]                      100.0
Total Fees Paid                      30516.809183
Max Drawdown [%]                         7.544795
Max Drawdown Duration            86 days 00:00:00
Total Trades                                   14
Total Closed Trades                            14
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            71.428571
Best Trade [%]                           7.195648
Worst Trade [%]                         -2.727795
Avg Winning Trade [%]                    2.782154
Avg Losing Trade [%]                    -1.728089


In [35]:
fig = portfolio.plot_orders()
fig.update_layout(template='plotly_dark', width=1200)
fig.show()

In [36]:
fig = portfolio.plot_trade_pnl()
fig.update_layout(template='plotly_dark', width=1200)
fig.show()